# Training

In [ ]:
import numpy as np
import tensorflow as tf
import cv2
import matplotlib.pyplot as plt

import os

## Defining paths

In [ ]:
research_path = '/content/drive/MyDrive/obj/Tensorflow/models/research/'
training_path = '/content/drive/MyDrive/obj/Tensorflow/workspace/Training/'
annot_path = '/content/drive/MyDrive/obj/Tensorflow/workspace/Training/annotations/' #labelmap, train.record, test.record
images_path = '/content/drive/MyDrive/obj/Tensorflow/workspace/Training/images/' #train, test
pre_trained_path = '/content/drive/MyDrive/obj/Tensorflow/workspace/Training/pre-trained-models/' # ssd_resnet50_v1_fpn_640x640_coco17_tpu-8
models_path = '/content/drive/MyDrive/obj/Tensorflow/workspace/Training/models/' # my_ssd_resnet50_v1_fpn
export_path = '/content/drive/MyDrive/obj/Tensorflow/workspace/Training/exported-models/'
root_path = '/content/drive/MyDrive/obj/'

# custom model details
mymodel_name = 'ssd_mobilenet_v2' #Change this to set new model name
model_used = mymodel_name + '/'  

#change the url for new pre-trained-model:
pretrain_dwd = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8.tar.gz'
pretrain_used = pretrain_dwd.split('/')[-1].split('.')[0] + '/' 


my_model_path = models_path + model_used
my_model_config = my_model_path + 'pipeline.config'



## Installing COCO API

In [ ]:
#compile all the proto buff files
#%cd {research_path}
#!protoc object_detection/protos/*.proto --python_out=.

#COCO API Installation
#%cd {root_path}
#!git clone https://github.com/cocodataset/cocoapi.git
#%cd cocoapi/PythonAPI
#!make

#!cp -r pycocotools {research_path}


## Installing Object Detection API and testing it.


In [ ]:
%cd {research_path}
!cp object_detection/packages/tf2/setup.py .
!python -m pip install --use-feature=2020-resolver .
!pip install avro-python3==1.9.1
!python install . #compile all files in the research folder
#test
!python object_detection/builders/model_builder_tf2_test.py

## Download the Pre-Trained Model and extract from the tar file.

In [ ]:
%cd {pre_trained_path}
pretrain_model = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8.tar.gz'
!wget pretrain_dwd
!tar -xvf ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8.tar.gz

## Create a new model folder in %models and Copy the pipeline file to %models folder

In [ ]:
!mkdir {my_model_path}

!cp {pre_trained_path+pretrain_used+'pipeline.config'} {my_model_path}

## Tweak the hyperparameters in pipeline config


In [ ]:
#after changing, copy model_main_tf2.py from object detection/research

!cp {research_path+'object_detection/model_main_tf2.py'} {training_path}

## Training: Run the model_main_tf2.py file
* The training produces checkpoints after a certain set of steps

In [ ]:
%cd {training_path}
!python model_main_tf2.py --model_dir={my_model_path} --pipeline_config_path={my_model_config}

## Evaluate
* Using the last checkpoint after our training

In [ ]:
!python model_main_tf2.py --model_dir=models/ssd_mobilenet_v2 --pipeline_config_path=models/ssd_mobilenet_v2/pipeline.config --checkpoint_dir=models/ssd_mobilenet_v2


## Exporting the trained model as saving it as .pb file

In [ ]:
#copying the exporter TF script and storing it in our training folder
!cp ../../models/research/object_detection/exporter_main_v2.py .
!python exporter_main_v2.py --input_type image_tensor --pipeline_config_path models/ssd_mobilenet_v2/pipeline.config --trained_checkpoint_dir models/ssd_mobilenet_v2 --output_directory exported_models/ssd_mobilenet_v2
